In [ ]:
import torch
import torch.optim as optim
from tqdm import tqdm
from utils.base_model import BiLSTM_CRF
from utils.utils import prepare, predict

In [ ]:
hparams = {
    'path':'/home/peitian_zhang/Data/NER/labeled_train.txt',
    'epochs': 150,
    'batch_size': 100,
    'embedding_dim': 300,
    'hidden_dim': 150,
    'device':'cuda:0',
}

In [ ]:
tag2idx, vocab, loader = prepare(hparams)
hparams['vocab_size'] = len(vocab)
hparams['seq_length'] = loader.dataset.max_length
tag2idx

In [ ]:
model = BiLSTM_CRF(hparams, tag2idx).to(hparams['device'])

In [ ]:
with torch.no_grad():
    record = next(iter(loader))
    _, tag_seq = model(record['token'])
    print("Prediction:{}\n Ground Truth:{}".format(tag_seq, record['label']))

In [ ]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
for epoch in range(hparams['epochs']):
    tqdm_ = tqdm(enumerate(loader))
    total_loss = 0

    for step,x in tqdm_:
        model.zero_grad()
        loss = model.neg_log_likelihood(x)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        tqdm_.set_description("epoch {:d} , step {:d} , loss: {:.4f}".format(epoch+1, step, total_loss/(step+1)))

In [ ]:
with torch.no_grad():
    record = next(iter(loader))
    _, tag_seq = model(record['token'])
    print("Prediction:{}\n Ground Truth:{}".format(tag_seq, record['label']))

In [ ]:
predict(['窦志成获奖'],model,vocab)